In [127]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from io import StringIO

import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.preprocessing import LabelEncoder

import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kelvin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load Model

In [118]:
import pickle
sentiment_model = pickle.load(open("E:/Research ML/LSTM_SentimentAnalysis.pkl", 'rb'))
sarcasm_model = pickle.load(open("E:/Research ML/LSTM_SarcasmDetection.pkl", 'rb'))

# Data Preprocessing

In [132]:
text = """text
I love this movie
"""

data = StringIO(text)
df = pd.read_csv(data)

In [125]:
df = pd.read_csv('E:\movie.csv')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Kelvin\AppData\Local\Temp\ipykernel_22512\837055456.py:1: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_csv('E:\movie.csv')


In [133]:
print(df)

                text
0  I love this movie


In [134]:
lab_to_sentiment = {0:"Negative", 1:"Positive"}
def label_decoder(label):
  return lab_to_sentiment[label]
df.sentiment = df.label.apply(lambda x: label_decoder(x))

AttributeError: 'DataFrame' object has no attribute 'label'

In [135]:
stemmer = SnowballStemmer('english')

def decontracted(phrase):
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

<>:13: SyntaxWarning: invalid escape sequence '\S'
<>:13: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Kelvin\AppData\Local\Temp\ipykernel_22512\2477235340.py:13: SyntaxWarning: invalid escape sequence '\S'
  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"


In [136]:
def preprocess(text, stem=False):
  text = decontracted(text)
  text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if stem:
      tokens.append(stemmer.stem(token))
    else:
      tokens.append(token)
  return " ".join(tokens)

In [137]:
df.text = df.text.apply(lambda x: preprocess(x))

In [139]:
val_count = df.sentiment.value_counts()

plt.figure(figsize=(8,4))
plt.bar(val_count.index, val_count.values)
plt.title("Sentiment Data Distribution")

AttributeError: 'DataFrame' object has no attribute 'sentiment'

In [140]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.text)

In [141]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
test_data = pad_sequences(tokenizer.texts_to_sequences(df.text), maxlen = 30)

# Before Sarcasm Detection

In [142]:
def decode_sentiment(score):
    return "Positive" if score>0.5 else "Negative"

scores = sentiment_model.predict(test_data, verbose=1, batch_size=10000)
before_pred_1d = [decode_sentiment(score) for score in scores]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step


In [143]:
print(before_pred_1d)

['Positive']


In [21]:
from sklearn.metrics import classification_report
print(classification_report(list(df.sentiment), before_pred_1d))

AttributeError: 'str' object has no attribute 'sentiment'

# Check Sarcasm

In [41]:
sarcasm_scores = sarcasm_model.predict(test_data, verbose=1, batch_size=1000)

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step


In [42]:
after_pred_1d = []

In [43]:
for idx, score in enumerate(scores):
    sentiment = ""
    if score>0.5:
        sentiment = "Positive"
    else:
        sentiment = "Negative"
        
    if sarcasm_scores[idx][0]>0.5 and sentiment == "Positive":
        sentiment = "Negative"
    elif sarcasm_scores[idx][0]>0.5 and sentiment == "Negative":
        sentiment = "Positive"
    
    after_pred_1d.append(sentiment)

# After Sarcasm Detection

In [44]:
from sklearn.metrics import classification_report
print(classification_report(list(df.sentiment), after_pred_1d))

              precision    recall  f1-score   support

    Negative       0.51      0.45      0.48     20019
    Positive       0.51      0.56      0.53     19981

    accuracy                           0.51     40000
   macro avg       0.51      0.51      0.50     40000
weighted avg       0.51      0.51      0.50     40000

